In [1]:
import pandas as pd
import nltk # Natural Language Toolkit
import re # Regular Expression
import string
import math
import numpy as np
import json

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

In [2]:
class PreProcessing():
    def PreProcess(dokumen):
        case = PreProcessing.Case_Folding(dokumen)
        english = PreProcessing.remove_english_words(case)
        token = PreProcessing.Tokenizing(english)
        stopword = PreProcessing.Stopword(english)
        stemming = PreProcessing.Stemming(stopword)
        return stemming
    
    def PreProcess2(dokumen):
        case = PreProcessing.Case_Folding(dokumen)
        english = PreProcessing.remove_english_words(case)
#         token = PreProcessing.Tokenizing(case)
        return english
        
    def Case_Folding(query):
        #folder = "Hadits Bukhari-Muslim/"
        #query = "hadits tentang"
        #data_hadits = tampil.Tampdbil_Hadis()
        #data_tes = open(folder + data_hadits[0][2], "r").read()
        lower_case = query.lower()
        removing_number = re.sub(r"\d+", "", lower_case)
        hapus_tanda_baca = removing_number.translate(str.maketrans("","",string.punctuation))
        hasil = hapus_tanda_baca.strip()
        return hasil

    def remove_english_words(doc):
        word_list = doc.split()
        english_vocab = set(words.words())
        filtered_words = [word for word in word_list if word.lower() not in english_vocab]
        filtered_english = ' '.join(filtered_words)
        return filtered_english

    def Tokenizing(doc):
        tes = doc.split(' ')
        return tes

    def Stopword(doc):
        stop_words = set(stopwords.words('indonesian'))
        stop_words.remove('bapak')
        #print(stop_words)
        tokens = word_tokenize(doc)
        filtered_tokens = [word for word in tokens if word.casefold() not in stop_words]
        filtered_text = ' '.join(filtered_tokens)
        return filtered_text

    def Stemming(doc):
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        doc = stemmer.stem(doc)
        return doc

In [3]:
def add_preprocess_to_df(data):

    preprocess = []
    preprocess2 = []
    preprocess_kalimat = []
    preprocess_kalimat2 = []
    for i in range(len(data)):
        preprocess.append(data['textpreprocessing'][i].split())
        isi = open("Hadits Bukhari-Muslim/"+data['file'][i], "r").read()
        preprocess2.append(PreProcessing.PreProcess2(isi).split(' '))
        isi = sent_tokenize(isi.lower())
        hasil_prepro = []
        hasil_prepro2 = []
        for k in range(len(isi)):
            hasil_prepro.append(PreProcessing.PreProcess(isi[k]))
            hasil_prepro2.append(PreProcessing.PreProcess2(isi[k]))
        isi = ""
        preprocess_kalimat.append(hasil_prepro)
        preprocess_kalimat2.append(hasil_prepro2)

    data['preprocess'] = preprocess
    data['preprocess2'] = preprocess2
    data['preprocess_kalimat'] = preprocess_kalimat
    data['preprocess_kalimat2'] = preprocess_kalimat2
    
    return data

In [4]:
data = pd.read_csv("tb_hadits.csv")
add_preprocess_to_df(data)
data

,id,judul_bab,file,textpreprocessing,preprocess,preprocess2,preprocess_kalimat,preprocess_kalimat2
0,1,Hadis Tentang Wahyu,1. Hadis Tentang Wahyu/1.txt,riwayat aisyah ibu orangorang iman bahwasanya ...,"[riwayat, aisyah, ibu, orangorang, iman, bahwa...","[diriwayatkan, aisyah, ibu, orangorang, berima...",[riwayat aisyah orangorang iman permualaan wah...,[diriwayatkan aisyah ibu orangorang beriman ba...
1,2,Hadis Tentang Wahyu,1. Hadis Tentang Wahyu/2.txt,riwayat aisyah ibu kaum mukminin alharrits bin...,"[riwayat, aisyah, ibu, kaum, mukminin, alharri...","[diriwayatkan, aisyah, ibu, kaum, mukminin, ba...",[riwayat aisyah kaum mukminin alharrits hisyam...,[diriwayatkan aisyah ibu kaum mukminin bahwa a...
2,3,"Hadis Tentang Akidah, Iman, dan Islam","2. Hadis Tentang Akidah, Iman, dan Islam/1.txt",riwayat abu hurairah radhiallahu anhu kata nab...,"[riwayat, abu, hurairah, radhiallahu, anhu, ka...","[diriwayatkan, hurairah, radhiallahu, anhu, be...",[riwayat hurairah radhiallahu anhu nabi shalla...,[diriwayatkan hurairah radhiallahu anhu berkat...
3,4,"Hadis Tentang Akidah, Iman, dan Islam","2. Hadis Tentang Akidah, Iman, dan Islam/2.txt",riwayat anas bin malik nabi shallallahu alaihi...,"[riwayat, anas, bin, malik, nabi, shallallahu,...","[diriwayatkan, anas, nabi, shallallahu, wassal...",[riwayat anas nabi shallallahu wassalam beliau...,[diriwayatkan anas nabi shallallahu wassalam b...
4,5,"Hadis Tentang Akidah, Iman, dan Islam","2. Hadis Tentang Akidah, Iman, dan Islam/3.txt",diriwiyatkan abu hurairah radhiallahu anhu kat...,"[diriwiyatkan, abu, hurairah, radhiallahu, anh...","[diriwiyatkan, hurairah, radhiallahu, anhu, di...",[diriwiyatkan hurairah radhiallahu anhu rasulu...,[diriwiyatkan hurairah radhiallahu anhu dia be...
...,...,...,...,...,...,...,...,...
276,277,Hadis Tentang Hak dan Kasih Sayang,29. Hadis Tentang Hak dan Kasih Sayang/8.txt,riwayat muadz radliallahu anhu berkataaku pern...,"[riwayat, muadz, radliallahu, anhu, berkataaku...","[diriwayatkan, muadz, radliallahu, anhu, ia, b...",[riwayat muadz radliallahu anhu berkataaku bon...,[diriwayatkan muadz radliallahu anhu ia berkat...
277,278,Hadis Tentang Hak dan Kasih Sayang,29. Hadis Tentang Hak dan Kasih Sayang/9.txt,riwayat abu hurairah radliallahu anhu kata ras...,"[riwayat, abu, hurairah, radliallahu, anhu, ka...","[diriwayatkan, hurairah, radliallahu, anhu, ia...",[riwayat hurairah radliallahu anhu rasulullah ...,[diriwayatkan hurairah radliallahu anhu ia ber...
278,279,Hadis Tentang Hak dan Kasih Sayang,29. Hadis Tentang Hak dan Kasih Sayang/10.txt,riwayat uqbah bin amr kata hudzaifahtidakkah k...,"[riwayat, uqbah, bin, amr, kata, hudzaifahtida...","[diriwayatkan, uqbah, amr, ia, berkata, kepada...",[riwayat uqbah amr hudzaifahtidakkah sedia cer...,[diriwayatkan uqbah amr ia berkata kepada hudz...
279,280,Hadis Tentang Ujian dan Kesabaran,30. Hadis Tentang Ujian dan Kesabaran/1.txt,riwayat aisyah radiallahuanha istri nabi shall...,"[riwayat, aisyah, radiallahuanha, istri, nabi,...","[diriwayatkan, aisyah, radiallahuanha, istri, ...",[riwayat aisyah radiallahuanha istri nabi shal...,[diriwayatkan aisyah radiallahuanha istri nabi...


In [5]:
class vsm():
    def process_vsm(data, query):
        document_term = vsm.term_document(data)
        idf_dokumen = vsm.idf_document(data)
        tf_idf_dokumen = vsm.tf_idf(document_term, idf_dokumen)
        dc_vectors = vsm.document_vector(tf_idf_dokumen)
        tfidf_query = vsm.tf_idf_query(idf_dokumen, query)
        query_vectors = vsm.q_vector(tfidf_query)
        scores = vsm.cosine_similarity(data, tfidf_query, tf_idf_dokumen, query_vectors, dc_vectors)
        
        return scores
        
    def term_document(data):
        document_term = []
        for document in data['preprocess']:
            #Calculating the term frequency 
            term_frequency = {}
            for term in document:
                if term in term_frequency:
                    term_frequency[term] += 1
                else:
                    term_frequency[term] = 1

            document_term.append(term_frequency)

        return document_term
    
    def idf_document(data):
        DF = {}
        for i in range(len(data['preprocess'])):
            tokens = data['preprocess'][i]
            for w in tokens:
                try:
                    # add token as key and doc number as value is chained
                    DF[w].add(i)
                except:
                    # to handle when a new token is encountered
                    DF[w] = {i}

        for i in DF:
            # convert to number of occurences of the token from list of documents where token occurs
            DF[i] = len(DF[i])

        idf_dokumen = {}
        for i in DF:
            idf_dokumen[i] = (math.log(len(data)/DF[i]))

        return idf_dokumen
    
    def tf_idf(document_term, idf_dokumen):
        tf_idf_dokumen = []
        for document in document_term:
            tf_idf = {}
            for term in document:
                if term in idf_dokumen:
                    tf_idf[term] = document[term]*idf_dokumen[term]
                    #print(document[term])

            tf_idf_dokumen.append(tf_idf)

        return tf_idf_dokumen
    
    def document_vector(tf_idf_dokumen):
        dc_vectors = []
        for document in tf_idf_dokumen:
            jumlah = 0
            for term in document.values():
                #print(term)
                jumlah = jumlah + (term*term)

            dc_vectors.append(math.sqrt(jumlah))

        return dc_vectors
    
    def tf_idf_query(idf_dokumen, query: str) -> list:
        '''
        preprocess the query
        :param query: the query
        '''
        query_tf = {}
        for term in query:
            if term in query_tf:
                query_tf[term] += 1
            else:
                query_tf[term] = 1

        q_tf_idf = {}
        for term in query_tf:
            if term in idf_dokumen:
                #print(term)
                q_tf_idf[term] = query_tf[term]*idf_dokumen[term]

        return q_tf_idf
    
    def q_vector(tfidf_query):
        query_vector = sum([freq**2 for freq in tfidf_query.values()])
        query_vector = math.sqrt(query_vector)

        return query_vector
    
    def cosine_similarity(data, tfidf_query, tf_idf_dokumen, query_vector, dc_vectors):
        scores = []
        for i in range(len(data)):
            tfidf = 0
            for term in tfidf_query:
                if term in tf_idf_dokumen[i]:\
                    tfidf = tfidf + (tfidf_query[term]*tf_idf_dokumen[i][term])

            scores.append(tfidf/(query_vector*dc_vectors[i]))

        return scores

In [6]:
class vsm2():
    def process_vsm(data, query):
        document_term = vsm2.term_document(data)
        idf_dokumen = vsm2.idf_document(data)
        tf_idf_dokumen = vsm2.tf_idf(document_term, idf_dokumen)
        dc_vectors = vsm2.document_vector(tf_idf_dokumen)
        tfidf_query = vsm2.tf_idf_query(idf_dokumen, query)
        query_vectors = vsm2.q_vector(tfidf_query)
        scores = vsm2.cosine_similarity(data, tfidf_query, tf_idf_dokumen, query_vectors, dc_vectors)
        
        return scores
        
    def term_document(data):
        document_term = []
        for document in data['preprocess2']:
            #Calculating the term frequency 
            term_frequency = {}
            for term in document:
                if term in term_frequency:
                    term_frequency[term] += 1
                else:
                    term_frequency[term] = 1

            document_term.append(term_frequency)

        return document_term
    
    def idf_document(data):
        DF = {}
        for i in range(len(data['preprocess2'])):
            tokens = data['preprocess2'][i]
            for w in tokens:
                try:
                    # add token as key and doc number as value is chained
                    DF[w].add(i)
                except:
                    # to handle when a new token is encountered
                    DF[w] = {i}

        for i in DF:
            # convert to number of occurences of the token from list of documents where token occurs
            DF[i] = len(DF[i])

        idf_dokumen = {}
        for i in DF:
            idf_dokumen[i] = (math.log(len(data)/DF[i]))

        return idf_dokumen
    
    def tf_idf(document_term, idf_dokumen):
        tf_idf_dokumen = []
        for document in document_term:
            tf_idf = {}
            for term in document:
                if term in idf_dokumen:
                    tf_idf[term] = document[term]*idf_dokumen[term]
                    #print(document[term])

            tf_idf_dokumen.append(tf_idf)

        return tf_idf_dokumen
    
    def document_vector(tf_idf_dokumen):
        dc_vectors = []
        for document in tf_idf_dokumen:
            jumlah = 0
            for term in document.values():
                #print(term)
                jumlah = jumlah + (term*term)

            dc_vectors.append(math.sqrt(jumlah))

        return dc_vectors
    
    def tf_idf_query(idf_dokumen, query: str) -> list:
        '''
        preprocess the query
        :param query: the query
        '''
        query_tf = {}
        for term in query:
            if term in query_tf:
                query_tf[term] += 1
            else:
                query_tf[term] = 1

        q_tf_idf = {}
        for term in query_tf:
            if term in idf_dokumen:
                #print(term)
                q_tf_idf[term] = query_tf[term]*idf_dokumen[term]

        return q_tf_idf
    
    def q_vector(tfidf_query):
        query_vector = sum([freq**2 for freq in tfidf_query.values()])
        query_vector = math.sqrt(query_vector)

        return query_vector
    
    def cosine_similarity(data, tfidf_query, tf_idf_dokumen, query_vector, dc_vectors):
        scores = []
        for i in range(len(data)):
            tfidf = 0
            for term in tfidf_query:
                if term in tf_idf_dokumen[i]:\
                    tfidf = tfidf + (tfidf_query[term]*tf_idf_dokumen[i][term])

            scores.append(tfidf/(query_vector*dc_vectors[i]))

        return scores

In [7]:
def load(filename):
    with open(filename) as data_file:
        data = json.load(data_file)
    
    return data

# load dictionary
mydict = load('dict.json')

def getSinonim(word):
    if word in mydict.keys():
        return mydict[word]['sinonim']
    else:
        return []

def expand_query(query):
    query_expand = []
    for term in query:
        query_expand.append(term)
        query_expand = query_expand + getSinonim(term)

    query_expand = ' '.join([str(elem) for elem in query_expand])
    query_expand = PreProcessing.PreProcess(query_expand)
    query_expand = re.split('\W+', query_expand)
    
    return query_expand

def expand_query2(query):
    query_expand = []
    for term in query:
        query_expand.append(term)
        query_expand = query_expand + getSinonim(term)

#     query_expand = ' '.join([str(elem) for elem in query_expand])
#     query_expand = PreProcessing.PreProcess(query_expand)
#     query_expand = re.split('\W+', query_expand)
    
    return query_expand


<>:24: DeprecationWarning: invalid escape sequence \W
<>:24: DeprecationWarning: invalid escape sequence \W
C:\Users\Rz\AppData\Local\Temp/ipykernel_19828/1562428229.py:24: DeprecationWarning: invalid escape sequence \W
  query_expand = re.split('\W+', query_expand)


In [8]:
class expand_query_fp_growth():
    def process(data, dokumen_teratas, query, min_support, min_threshold):
        data_master = expand_query_fp_growth.add_data_master_fp_growth(data, dokumen_teratas)
        # data_master_fp_growth = add_data_master_fp_growth(data, dokumen_teratas)
        my_transactionencoder = TransactionEncoder()

        # fit the transaction encoder using the list of transaction tuples
        my_transactionencoder.fit(data_master)

        # transform the list of transaction tuples into an array of encoded transactions
        encoded_transactions = my_transactionencoder.transform(data_master)

        # convert the array of encoded transactions into a dataframe
        encoded_transactions_df = pd.DataFrame(encoded_transactions, columns=my_transactionencoder.columns_)
        
#         min_support = 0.2
        # compute the frequent itemsets using fpgriowth from mlxtend
        from mlxtend.frequent_patterns.fpgrowth import fpgrowth
        frequent_itemsets = fpgrowth(encoded_transactions_df, min_support=min_support, use_colnames = True)
        
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_threshold)
        if(rules == None):
            message = "Data yang digunakan tidak ditemukan rulesnya"
            return message
        
        df_rules = pd.DataFrame(rules)
        df_rules = df_rules.sort_values(['confidence', 'lift'], ascending=False)
        
        expand = []
        for term in query:
            if expand_query_fp_growth.query_expand(term, df_rules) != None:
                expand.append(expand_query_fp_growth.query_expand(term, df_rules))
                
        return expand
    
    def add_data_master_fp_growth(data, dokumen_teratas):
        data_master_fp_growth = []
        for term in dokumen_teratas:
            for kalimat in data['preprocess_kalimat'][term]:
        #     for kalimat in eval(data['textpreprocessing_kalimat'][term]):
                data_master_fp_growth.append(kalimat.split())

        return data_master_fp_growth
    
    def query_expand(term, df_rules):
        for i in range(len(df_rules)):
            #print(list(df_rules['antecedents'].values[i])[0])
            #print(term)
            if len(list(df_rules['antecedents'].values[i])) > 1:
                for j in range(len(list(df_rules['antecedents'].values[i]))):
                    if term == list(df_rules['antecedents'].values[i])[j]:
                        #print(j)
                        init = list(df_rules['consequents'].values[i])[0]
                        return init
            else:
                if term == list(df_rules['antecedents'].values[i])[0]:
                    #print(1)
                    init = list(df_rules['consequents'].values[i])[0]
                    return init

In [9]:
class expand_query_fp_growth2():
    def process(data, dokumen_teratas, query, min_support, min_threshold):
        data_master = expand_query_fp_growth2.add_data_master_fp_growth(data, dokumen_teratas)
        # data_master_fp_growth = add_data_master_fp_growth(data, dokumen_teratas)
        my_transactionencoder = TransactionEncoder()

        # fit the transaction encoder using the list of transaction tuples
        my_transactionencoder.fit(data_master)

        # transform the list of transaction tuples into an array of encoded transactions
        encoded_transactions = my_transactionencoder.transform(data_master)

        # convert the array of encoded transactions into a dataframe
        encoded_transactions_df = pd.DataFrame(encoded_transactions, columns=my_transactionencoder.columns_)
        
#         min_support = 0.2
        # compute the frequent itemsets using fpgriowth from mlxtend
        from mlxtend.frequent_patterns.fpgrowth import fpgrowth
        frequent_itemsets = fpgrowth(encoded_transactions_df, min_support=min_support, use_colnames = True)
        
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_threshold)
        
        df_rules = pd.DataFrame(rules)
        df_rules = df_rules.sort_values(['confidence', 'lift'], ascending=False)
        
        expand = []
        for term in query:
            if expand_query_fp_growth2.query_expand(term, df_rules) != None:
                expand.append(expand_query_fp_growth2.query_expand(term, df_rules))
                
        return expand
    
    def add_data_master_fp_growth(data, dokumen_teratas):
        data_master_fp_growth = []
        for term in dokumen_teratas:
            for kalimat in data['preprocess_kalimat2'][term]:
        #     for kalimat in eval(data['textpreprocessing_kalimat'][term]):
                data_master_fp_growth.append(kalimat.split())

        return data_master_fp_growth
    
    def query_expand(term, df_rules):
        for i in range(len(df_rules)):
            #print(list(df_rules['antecedents'].values[i])[0])
            #print(term)
            if len(list(df_rules['antecedents'].values[i])) > 1:
                for j in range(len(list(df_rules['antecedents'].values[i]))):
                    if term == list(df_rules['antecedents'].values[i])[j]:
                        #print(j)
                        init = list(df_rules['consequents'].values[i])[0]
                        return init
            else:
                if term == list(df_rules['antecedents'].values[i])[0]:
                    #print(1)
                    init = list(df_rules['consequents'].values[i])[0]
                    return init

In [32]:
from IPython.display import clear_output

while True:
    clear_output(wait=True)
    pilihan = input('1.VSM\n2.Sinonim + VSM\n3.VSM + FP-Growth\n4.Sinonim + VSM + FP-Growth\n5.Exit\nMasukkan Pilihan :')
    if pilihan == '1':
        query = input('Masukkan query:')
        query_prepro = PreProcessing.PreProcess(query)
        query_prepro = re.split('\W+', query_prepro)
        query = PreProcessing.PreProcess2(query)
        query = re.split('\W+', query)
#         print(query_prepro)
        score_vsm = vsm.process_vsm(data, query_prepro)
        score_vsm2 = vsm2.process_vsm(data, query)
        print('Query Menggunakan Preprocess : ', ' '.join([str(elem) for elem in query_prepro]))
        print('Hasil VSM Menggunakan Preprocess')
        dokumen_teratas = sorted(range(len(score_vsm)), key=lambda i: score_vsm[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[dokumen_teratas[i]]['file'])
            print('File: ', data.iloc[dokumen_teratas[i]]['file'])
            print('Score: ', score_vsm[dokumen_teratas[i]], end="\n---------------XXX---------------")
        
        print('\n')
        print('Query tanpa Preprocess: ', ' '.join([str(elem) for elem in query]))
        print('Hasil VSM tanpa Preprocess')
        dokumen_teratas2 = sorted(range(len(score_vsm2)), key=lambda i: score_vsm2[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[dokumen_teratas2[i]]['file'])
            print('File: ', data.iloc[dokumen_teratas2[i]]['file'])
            print('Score: ', score_vsm2[dokumen_teratas2[i]], end="\n---------------XXX---------------")
            
        lanjut = input("Apakah anda ingin mencoba menu lain ? (y/n)")
        if lanjut == 'y':
            clear_output(wait=True)
        else:
            break
    elif pilihan == '2':
        query = input('Masukkan query:')
        query_prepro = PreProcessing.PreProcess(query)
        query_prepro = re.split('\W+', query_prepro)
        query_prepro = expand_query(query_prepro)
        score_vsm = vsm.process_vsm(data, query_prepro)
        print('Query Menggunakan Preprocess Setelah diExpand: ', ' '.join([str(elem) for elem in query_prepro]))
        print('Hasil VSM Menggunakan Preprocess')
        dokumen_teratas = sorted(range(len(score_vsm)), key=lambda i: score_vsm[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[dokumen_teratas[i]]['file'])
            print('File: ', data.iloc[dokumen_teratas[i]]['file'])
            print('Score: ', score_vsm[dokumen_teratas[i]], end="\n---------------XXX---------------")
        
        query = PreProcessing.PreProcess2(query)
        query = re.split('\W+', query)
        query = expand_query2(query)
        score_vsm2 = vsm2.process_vsm(data, query)    
        print('\n')
        print('Query tanpa Preprocess Setelah diExpand: ', ' '.join([str(elem) for elem in query]))
        print('Hasil VSM tanpa Preprocess')
        dokumen_teratas2 = sorted(range(len(score_vsm2)), key=lambda i: score_vsm2[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[dokumen_teratas2[i]]['file'])
            print('File: ', data.iloc[dokumen_teratas2[i]]['file'])
            print('Score: ', score_vsm2[dokumen_teratas2[i]], end="\n---------------XXX---------------")
            
        lanjut = input("Apakah anda ingin mencoba menu lain ? (y/n)")
        if lanjut == 'y':
            clear_output(wait=True)
        else:
            break
    elif pilihan == '3':
        query = input('Masukkan query:')
        query_prepro = PreProcessing.PreProcess(query)
        query_prepro = re.split('\W+', query_prepro)
        score_vsm = vsm.process_vsm(data, query_prepro)
        print('Query yang Menggunakan Preprocess: ', ' '.join([str(elem) for elem in query_prepro]))
        print('5 Dokumen Teratas yang dijadikan data master FP-Growth')            
        dokumen_teratas = sorted(range(len(score_vsm)), key=lambda i: score_vsm[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[dokumen_teratas[i]]['file'])
            print('File: ', data.iloc[dokumen_teratas[i]]['file'])
            print('Score: ', score_vsm[dokumen_teratas[i]], end="\n---------------XXX---------------")

        eq_fp_growth = expand_query_fp_growth.process(data, dokumen_teratas, query_prepro, 0.1, 0.5)
        query_expand = query_prepro + eq_fp_growth
        query_expand = ' '.join([str(elem) for elem in query_expand])
        
        print('\n')
        print('\nQuery setelah diExpand menggunakan FP-Growth  : ', query_expand)
        query_expand = re.split('\W+', query_expand)
        score_akhir = vsm.process_vsm(data, query_expand)
        print('Hasil VSM menggunakan Preprocess')
        hasil = sorted(range(len(score_akhir)), key=lambda i: score_akhir[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[hasil[i]]['file'])
            print('File: ', data.iloc[hasil[i]]['file'])
            print('Score: ', score_akhir[hasil[i]], end="\n---------------XXX---------------")
        
        query = PreProcessing.PreProcess2(query)
        query = re.split('\W+', query)
        score_vsm2 = vsm2.process_vsm(data, query)    
        print('\n')
        print('Query tanpa Preprocess : ', ' '.join([str(elem) for elem in query]))
        print('5 Dokumen Teratas yang dijadikan data master FP-Growth')
        dokumen_teratas2 = sorted(range(len(score_vsm2)), key=lambda i: score_vsm2[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[dokumen_teratas2[i]]['file'])
            print('File: ', data.iloc[dokumen_teratas2[i]]['file'])
            print('Score: ', score_vsm2[dokumen_teratas2[i]], end="\n---------------XXX---------------")
        
        eq_fp_growth2 = expand_query_fp_growth2.process(data, dokumen_teratas, query, 0.1, 0.5)
        query_expand2 = query + eq_fp_growth2
        query_expand2 = ' '.join([str(elem) for elem in query_expand2])
        print('\n')
        print('\nQuery setelah diExpand menggunakan FP-Growth  : ', query_expand2)
        query_expand2 = re.split('\W+', query_expand2)
        score_akhir2 = vsm2.process_vsm(data, query_expand2)
        print('Hasil VSM tanpa Preprocess')
        hasil2 = sorted(range(len(score_akhir2)), key=lambda i: score_akhir2[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[hasil2[i]]['file'])
            print('File: ', data.iloc[hasil2[i]]['file'])
            print('Score: ', score_akhir[hasil2[i]], end="\n---------------XXX---------------")
            
        lanjut = input("Apakah anda ingin mencoba menu lain ? (y/n)")
        if lanjut == 'y':
            clear_output(wait=True)
        else:
            break
    elif pilihan == '4':
        query = input('Masukkan query:')
        query_prepro = PreProcessing.PreProcess(query)
        query_prepro = re.split('\W+', query_prepro)
        query_prepro = expand_query(query_prepro)
        score_vsm = vsm.process_vsm(data, query_prepro)
        print('Query yang Menggunakan Preprocess: ', ' '.join([str(elem) for elem in query_prepro]))
        print('5 Dokumen Teratas yang dijadikan data master FP-Growth')            
        dokumen_teratas = sorted(range(len(score_vsm)), key=lambda i: score_vsm[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[dokumen_teratas[i]]['file'])
            print('File: ', data.iloc[dokumen_teratas[i]]['file'])
            print('Score: ', score_vsm[dokumen_teratas[i]], end="\n---------------XXX---------------")

        eq_fp_growth = expand_query_fp_growth.process(data, dokumen_teratas, query_prepro, 0.1, 0.5)
        query_expand = query_prepro + eq_fp_growth
        query_expand = ' '.join([str(elem) for elem in query_expand])
        
        print('\n')
        print('\nQuery setelah diExpand menggunakan FP-Growth  : ', query_expand)
        query_expand = re.split('\W+', query_expand)
        score_akhir = vsm.process_vsm(data, query_expand)
        print('Hasil VSM menggunakan Preprocess')
        hasil = sorted(range(len(score_akhir)), key=lambda i: score_akhir[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[hasil[i]]['file'])
            print('File: ', data.iloc[hasil[i]]['file'])
            print('Score: ', score_akhir[hasil[i]], end="\n---------------XXX---------------")
        
        query = PreProcessing.PreProcess2(query)
        query = re.split('\W+', query)
        query = expand_query2(query)
        score_vsm2 = vsm2.process_vsm(data, query)    
        print('\n')
        print('Query tanpa Preprocess : ', ' '.join([str(elem) for elem in query]))
        print('5 Dokumen Teratas yang dijadikan data master FP-Growth')
        dokumen_teratas2 = sorted(range(len(score_vsm2)), key=lambda i: score_vsm2[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[dokumen_teratas2[i]]['file'])
            print('File: ', data.iloc[dokumen_teratas2[i]]['file'])
            print('Score: ', score_vsm2[dokumen_teratas2[i]], end="\n---------------XXX---------------")
        
        eq_fp_growth2 = expand_query_fp_growth2.process(data, dokumen_teratas, query, 0.1, 0.5)
        query_expand2 = query + eq_fp_growth2
        query_expand2 = ' '.join([str(elem) for elem in query_expand2])
        print('\n')
        print('\nQuery setelah diExpand menggunakan FP-Growth  : ', query_expand2)
        query_expand2 = re.split('\W+', query_expand2)
        score_akhir2 = vsm2.process_vsm(data, query_expand2)
        print('Hasil VSM tanpa Preprocess')
        hasil2 = sorted(range(len(score_akhir2)), key=lambda i: score_akhir2[i], reverse=True)[:5]
        for i in range(5):
            print('\n' + str(i+1) + ')\nTitle: ', data.iloc[hasil2[i]]['file'])
            print('File: ', data.iloc[hasil2[i]]['file'])
            print('Score: ', score_akhir[hasil2[i]], end="\n---------------XXX---------------")
            
        lanjut = input("Apakah anda ingin mencoba menu lain ? (y/n)")
        if lanjut == 'y':
            clear_output(wait=True)
        else:
            break
    else:
        break

1.VSM
2.Sinonim + VSM
3.VSM + FP-Growth
4.Sinonim + VSM + FP-Growth
5.Exit
Masukkan Pilihan :4
Masukkan query:hadits tentang bapak
Query yang Menggunakan Preprocess:  hadits bapak abah ayahanda bapanda bokap buya empek kepala keluarga papi rama ramanda arsitek kreator cipta diri
5 Dokumen Teratas yang dijadikan data master FP-Growth

1)
Title:  17. Hadis Tentang Perintah dan Larangan/29.txt
File:  17. Hadis Tentang Perintah dan Larangan/29.txt
Score:  0.22397003754552214
---------------XXX---------------
2)
Title:  25. Hadis Tentang Penyakit dan Obat/5.txt
File:  25. Hadis Tentang Penyakit dan Obat/5.txt
Score:  0.1446831669028331
---------------XXX---------------
3)
Title:  17. Hadis Tentang Perintah dan Larangan/10.txt
File:  17. Hadis Tentang Perintah dan Larangan/10.txt
Score:  0.11810284629647426
---------------XXX---------------
4)
Title:  10. Hadis Tentang Jihad/16.txt
File:  10. Hadis Tentang Jihad/16.txt
Score:  0.11489718901046829
---------------XXX---------------
5)
Title:  